## Big Data Progrmming - Assignment 4

In [ ]:
#Importing packages
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import us
from mpl_toolkits.basemap import Basemap #conda install basemap if not installed. Also, conda install pillow for Python 3

In [ ]:
%%time
URL1 = "https://earthquake.usgs.gov/fdsnws/event/1/query?"
URL2 = "https://earthquake.usgs.gov/fdsnws/event/1/count?"

PARAMETERS1 = { "format": 'csv', "starttime": '2016-01-01', "endtime": '2017-01-01', "minmagnitude": 4.0}
PARAMETERS2 = { "format": 'csv', "starttime": '2017-01-01', "endtime": '2018-01-01', "minmagnitude": 4.0}
PARAMETERS3 = { "format": 'csv', "starttime": '2018-01-01', "endtime": '2019-01-01', "minmagnitude": 4.0}
PARAMETERS4 = { "format": 'csv', "starttime": '2019-01-01', "endtime": '2019-10-02', "minmagnitude": 4.0}

c1 = int(requests.get(url = URL2, params = PARAMETERS1).text)
c2 = int(requests.get(url = URL2, params = PARAMETERS2).text)
c3 = int(requests.get(url = URL2, params = PARAMETERS3).text)
c4 = int(requests.get(url = URL2, params = PARAMETERS4).text)

In [ ]:
%%time
r1 = requests.get(url = URL1, params = PARAMETERS1).text
r2 = requests.get(url = URL1, params = PARAMETERS2).text
r3 = requests.get(url = URL1, params = PARAMETERS3).text
r4 = requests.get(url = URL1, params = PARAMETERS4).text

In [ ]:
d = r1 + r2 + r3 + r4

In [ ]:
with open("data.txt", "w") as output:
    output.write(str(d))

In [ ]:
data = pd.read_csv("data.txt", sep=",", encoding ='latin1')
data = data[data.latitude !='latitude'] #Removing the rows having coulmn names as an entry
data = data[data.type == 'earthquake'] #Removing non-earthquake entries

#### Use describe to get the basic statistics of all the columns (5 points)

In [ ]:
data.describe()

#### Get the top 10 earthquakes by magnitude

In [ ]:
data['mag'] =pd.to_numeric(data['mag'])
data.nlargest(10, columns = ['mag'])

#### Handle all Null/empty data by filling it with zeros (10 points)

In [ ]:
df = data.fillna(0)
df
df.isna().sum()

#### Find the top 10 places where the strongest earthquakes occurred (15 points) (Note: Place needs to be parsed nicely to remove the KM location from them. For example: 75km WSW of Illapel, Chile should look like Illapel, Chile).

In [ ]:
q4 = df.nlargest(15, columns = ['mag'])
q4['Place'] = q4['place'].str.split('of').str[1]
q4.groupby(['Place',], sort=False)['mag'].max().head(10)

#### Find the top 10 places where the weakest earthquakes occurred (15 points) (Note: Place needs to be parsed nicely to remove the KM location from them. For example: 75km WSW of Illapel, Chile should look like Illapel, Chile)

In [ ]:
q5 = df.nsmallest(15, columns = ['mag'])
q5['Place'] = q5['place'].str.split('of').str[1]
q5.groupby(['Place'], sort=False)['mag'].max().head(10)

##### On a per-year basis, use a bar chart to plot the number of earthquakes for each of the following magnitude groups ranges: Group 1: [4,4.5), Group 2: [4.5,5), Group 3: [5,6), Group 4: [6,7), Group 5:(7,MAX]. Pay close attention to the group ranges. (20 points) Please add labels and colors to the plot.

In [ ]:
q6 = df[df.mag != 7.0 ] #Removing earthquakes with magnitude 7 as they are not needed
q6['time'] = pd.to_datetime(q6.time)
q6['Year'] = q6['time'].dt.year
q6['Group'] = pd.cut(q6['mag'], bins=[4,4.5,5,6,7,9],right=False, include_lowest = True, labels=["Group 1", "Group 2", "Group 3", "Group 4", "Group 5"])

plt.figure()
plt.suptitle('Year VS Earthquake count for every group')
plt.subplots_adjust(hspace=1, wspace=1)
plt.subplot(2, 2, 1)
plt.title("Year 2016")
plt.xlabel("Count")
plt.ylabel("Group")
q6_2016 = q6[q6.Year == 2016]
q6_2016.Group.value_counts().plot(kind='barh',color='aqua')
plt.subplot(2, 2, 2)
plt.title("Year 2017")
plt.xlabel("Count")
plt.ylabel("Group")
q6_2017 = q6[q6.Year == 2017]
q6_2017.Group.value_counts().plot(kind='barh', color = 'coral')
plt.subplot(2, 2, 3)
plt.title("Year 2018")
plt.xlabel("Count")
plt.ylabel("Group")
q6_2018 = q6[q6.Year == 2018]
q6_2018.Group.value_counts().plot(kind='barh', color = 'orchid')
plt.subplot(2, 2, 4)
plt.title("Year 2019")
plt.xlabel("Count")
plt.ylabel("Group")
q6_2019 = q6[q6.Year == 2019]
q6_2019.Group.value_counts().plot(kind='barh', color = 'yellowgreen')

##### Find the 10 countries with the highest number of earthquakes (30 points) (Note: Yes, this is only countries, not full place)

In [ ]:
q7 = df.copy()
def my_function(item1):
    item = str(item1)
    if(',' in item):
        item = item.split(', ')[1]
    elif('of' in item):
        item = item.split('of ')[1]
    else:
        item = item1
    return item

q7['Country'] = q6.place.apply(lambda x: my_function(x))
q7.groupby(q7.Country).size().nlargest(10)

##### Analyze the distribution of the Earthquake magnitudes. This is, make a histogram of the Earthquake count versus magnitude. Make sure to use a Logarithmic scale. What sort of relationship do you see? (20 points) Please add labels and colors to the plot.
                             

In [ ]:
plt.title("Earthquake count(log) VS Magnitude")
plt.xlabel("Magnitude")
plt.ylabel("Count(log)")
plt.hist(df.mag, log=True, color = 'violet');

Observation: A strong negative correlation between the log(number of earthquakes) and the magnitude. As the magnitude is increasing, the number of earthquakes is decreasing. Being a logarithmic scale, the decay is exponential. 

##### Analyze the distribution of the Earthquake depths. This is, make a histogram of the Earthquake count versus depth. Make sure to use a Logarithmic scale. What sort of relationship do you see? (20 points) Please add labels and colors to the plot.

In [ ]:
df['depth'] = pd.to_numeric(df['depth'])
plt.title("Earthquake count(log) VS Depth")
plt.xlabel("Depth(Km)")
plt.ylabel("Count(log)")
plt.hist(df.depth,bins=10, color='green')
plt.yscale('log')
plt.show()

Observation: A lot number of earthquakes originate from 0-70Km depth. The count decreases as we move deeper into earth's core till approx 270Km. Now, the earthquake surprisingly remains constant till 490Km. Between 490-610Km, the count increases rapidly and then drop till 700Km. 

##### Visualize the locations of earthquakes by making a scatterplot of their latitude and longitude. (20 points) Please add labels and colors to the plot.

In [ ]:
df['latitude'] = pd.to_numeric(df['latitude'])
df['longitude'] = pd.to_numeric(df['longitude'])
plt.title("Scatter plot of Earthquakes by Longitude and Latitude")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.scatter(df.longitude, df.latitude)

##### Using the US package (https://pypi.org/project/us/), clean the dataset you used previously to only have data from the USA . You need to create a function that accommodates this. (20 points)

In [ ]:
USA = [str(item) for item in us.STATES] #Considering only US states. Not all governing territories
q11 = df.copy()
def checkUSA(place, USA = USA):
    a = ''
    for item in USA:
        if item.lower() in place.lower():
            a = item
    return(a)
q11['State'] = q11.place.apply(lambda x: checkUSA(x, USA))
q11['State'].value_counts()
q11USA = q11[q11.State != '']

#Removing Earthquakes from 'South Georgia and the South Sandwich Islands' which were considered as Georgia state, USA
def georgia(item1):
    item = str(item1)
    if('South Georgia and the South Sandwich Islands' in item):
        item = 'Drop'
    if('South Georgia Island region' in item):
        item ='Drop'
    return item
q11USA['Drop'] = q11USA.place.apply(lambda x: georgia(x))
usDATA = q11USA[q11USA.Drop != 'Drop']

##### Find the top 10 US states where the strongest earthquakes occurred

In [ ]:
q11largest = usDATA.nlargest(300, columns = ['mag'])
q11largest.groupby(['State'], sort=False)['mag'].max().head(10)

##### On a per-year basis, use a bar chart to plot the number of earthquakes for each of the following magnitude groups ranges: Group 1: [4,4.5), Group 2: [4.5,5), Group 3: [5,6), Group 4: [6,7), Group 5: (7,MAX].

In [ ]:
q12USA = usDATA[usDATA.mag != 7.0 ] #Removing all entries which has earthquake magnitude = 7
q12USA['time'] = pd.to_datetime(q12USA.time)
q12USA['Year'] = q12USA['time'].dt.year
q12USA['Group'] = pd.cut(q12USA['mag'], bins=[4,4.5,5,6,7,9],right=False, include_lowest = True, labels=["Group 1", "Group 2", "Group 3", "Group 4", "Group 5"])

plt.figure()
plt.suptitle('Year VS Earthquake count for every group in United States of America')
plt.subplots_adjust(hspace=1, wspace=1)
plt.subplot(2, 2, 1)
plt.title("Year 2016")
plt.xlabel("Count")
plt.ylabel("Group")
q12USA_2016 = q12USA[q12USA.Year == 2016]
q12USA_2016.Group.value_counts().plot(kind='barh',color='aqua')
plt.subplot(2, 2, 2)
plt.title("Year 2017")
plt.xlabel("Count")
plt.ylabel("Group")
q12USA_2017 = q12USA[q12USA.Year == 2017]
q12USA_2017.Group.value_counts().plot(kind='barh', color = 'coral')
plt.subplot(2, 2, 3)
plt.title("Year 2018")
plt.xlabel("Count")
plt.ylabel("Group")
q12USA_2018 = q12USA[q12USA.Year == 2018]
q12USA_2018.Group.value_counts().plot(kind='barh', color = 'orchid')
plt.subplot(2, 2, 4)
plt.title("Year 2019")
plt.xlabel("Count")
plt.ylabel("Group")
q12USA_2019 = q12USA[q12USA.Year == 2019]
q12USA_2019.Group.value_counts().plot(kind='barh', color = 'yellowgreen')

##### Visualize the locations of earthquakes by making a scatterplot of their latitude and longitude. Overlay a US map on top of this plot to match the locations.

In [ ]:
usDATA['latitude'] = pd.to_numeric(usDATA['latitude'])
usDATA['longitude'] = pd.to_numeric(usDATA['longitude'])
fig = plt.gcf()
fig.set_size_inches(8, 8)
m = Basemap(projection='merc', \
            llcrnrlat=14, urcrnrlat=66, \
            llcrnrlon=-163, urcrnrlon=-50, \
            lat_ts=20, \
            resolution='c')
latitude = usDATA["latitude"].tolist()
longitude = usDATA["longitude"].tolist()

m.bluemarble(scale=0.5)
m.drawcoastlines(color='white', linewidth=0.2)
plt.title('Earthquakes in USA from 2016 to October 1, 2019')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
x, y = m(longitude, latitude)
plt.scatter(x, y, 10, marker='x', color='yellow') 
plt.show()

#Source: https://jakevdp.github.io/PythonDataScienceHandbook/04.13-geographic-data-with-basemap.html